In [1]:
# ライブラリ読み込み
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pronto
import copy
import networkx as nx
import os
import pickle
import glob
import gzip
from tqdm.notebook import tqdm
import itertools
#from collections import OrderedDict
#import collections
#from collections import Counter

In [2]:
#current
os.chdir("/home/admin_user/デスクトップ/UMLS2021AB")

#file_1 : pronto.Ontology.from_obo_library("hp.obo")
file_2 = "phenotype.hpoa"
file_3 = "CTD_diseases.tsv.gz"
file_4 = "MRFILES.RRF.gz"
file_5 = "MRSTY.RRF.gz"
file_6 = "MRCONSO.RRF.aa.gz"
file_7 = "MRCONSO.RRF.ab.gz"
file_8 = "SRFIL"

In [3]:
#HPOオントロジー

In [4]:
# 'hp.obo'読み込み　タームを取り出す
ont = pronto.Ontology.from_obo_library("hp.obo")
print(len(ont))

/home/admin_user/anaconda3/lib/python3.8/site-packages/pronto/ontology.py:206: UnicodeWarning: unsound encoding, assuming ISO-8859-1 (73% confidence)
  return cls(


16601


In [5]:
#HPOオントロジー中で XREFにUMLSを持つもの(UMLSへのマッピングの数)

hp=[]
umls = []
hp_umls=[]
hp_cui = []

for term in ont.terms():
#    if term.obsolete == False: #廃止タームを除く場合
        for each_xref in term.xrefs:
            if ("Xref('UMLS:" in str(each_xref)):
                cui = str(each_xref).replace("Xref('UMLS:", "").replace("')", "")
                hp.append(term.id) #HPO_id
                umls.append( cui) #CUI
                hp_umls.append([term.id, cui]) #HPO_id, CUI
                hp_cui.append([term.name + '(' + term.id + ')', term.id, cui]) #HPO_id, HPO_name, CUI
                
hp = sorted(list(set(hp)))
umls = sorted(list(set(umls)))
hp_umls = sorted(list(map(list, set(map(tuple, hp_umls)))))
hp_cui = sorted(list(map(list, set(map(tuple, hp_cui)))))

print("HPの数:", len(hp), "CUIの数:", len(umls), "マッピングの数:", len(hp_umls) ,len(hp_cui) )

#廃止タームを除外  HPの数: 11599 CUIの数: 12867 マッピングの数: 13016

HPの数: 11600 CUIの数: 12868 マッピングの数: 13017 13017


In [6]:
#'phenotype.hpoa'読み込み
print(file_2)
phenotype_df = pd.read_csv(file_2, header=4, sep="\t", engine="python")
#列名#DatabaseIDをOMIM に絞る
phenotype_df = phenotype_df[phenotype_df["#DatabaseID"].str.contains("OMIM")]
phenotype_df = phenotype_df.rename(columns={'#DatabaseID': "OMIM", "HPO_ID":"HP"})
phenotype_df = phenotype_df[["HP", "OMIM"]]
print("hpoaのアノテーション総数:", len(phenotype_df))
print("hpoaのOMIMの種類数:", phenotype_df["OMIM"].nunique())
print("hpoaのHPOの種類数:", phenotype_df["HP"].nunique())

phenotype.hpoa
hpoaのアノテーション総数: 120331
hpoaのOMIMの種類数: 8120
hpoaのHPOの種類数: 8041


In [7]:
#phenotype.hpoaにあるHPOのCUIマッピング

#XREFにUMLSを持つHPO
hp_df = pd.DataFrame(hp, columns=["HP"]) 
#phenotype_dfと合わせる
print("phenotype.hpoaにあるHPOからのCUIマッピング:", len(pd.merge(phenotype_df, hp_df)))
print("そのHPOの数:", pd.merge(phenotype_df, hp_df)["HP"].nunique())
print("そのOMIMの数:", pd.merge(phenotype_df, hp_df)["OMIM"].nunique())

phenotype.hpoaにあるHPOからのCUIマッピング: 117950
そのHPOの数: 7125
そのOMIMの数: 8120


In [8]:
#CTD

In [9]:
print(file_3) #2011検証と共通
all_ctd = pd.read_csv(file_3, sep='\t', header=27)
all_ctd = all_ctd[1:]
print("CTDレコード件数:", len(all_ctd))

#ctd = ctd.rename(columns={"# DiseaseName": "DiseaseName"}) #列名変更
ctd = all_ctd[["DiseaseID", "AltDiseaseIDs"]] #列を限定（→重複データなし ctd[~ctd.duplicated()]）
#DAltDiseaseIDsを持つものに限定
ctd = ctd[ctd["AltDiseaseIDs"].notna()]
ctd = ctd.reset_index(drop=True)
print("AltDiseaseIDsを持つレコード件数:", len(ctd))



CTD_diseases.tsv.gz
CTDレコード件数: 13144
AltDiseaseIDsを持つレコード件数: 6595


In [10]:
#DiseaseIDかAltDiseaseIDs に MESHかOMIMを持つレコードを探し、そのインデックス番号を取得

#「MeSH」
m_o_map_index = [ enu for enu, i in enumerate(ctd.values.tolist()) if (("MESH" in i[0]) & ("OMIM" in i[1])) | (("OMIM" in i[0]) & ("MESH" in i[1]))]

#インデックス番号でctdをサイズダウン
m_o_map = ctd.iloc[m_o_map_index]
m_o_map = m_o_map.reset_index(drop=True)
print(len(m_o_map))

#AltDiseaseIDs列のデータを要素に分割
m_o_map["AltDiseaseIDs"] = m_o_map["AltDiseaseIDs"].str.split("|")

3330


In [11]:
#m_o_mapは DiseaseID(MESH)：AltDiseaseIDs(他)＝１：n なので, DiseaseID(MESH)：AltDiseaseIDs(OMIM)=１：１に絞り込む
each_m_o_map = [ [m_o[0], each_id] for m_o in m_o_map.values.tolist() for each_id in m_o[1] if "OMIM" in each_id ]
each_m_o_map_df = pd.DataFrame(each_m_o_map, columns=["MESH", "OMIM"])
print("MESH")
print("MeSH-OMIMマッピングの数:", len(each_m_o_map_df))
print("MESHの数:", len(each_m_o_map_df["MESH"].unique()))
print("OMIMの数:", len(each_m_o_map_df["OMIM"].unique()))

MESH
MeSH-OMIMマッピングの数: 4051
MESHの数: 3330
OMIMの数: 4051


In [12]:
ctd = each_m_o_map_df

In [13]:
print("phenotype.hpoaのアノテーション総数:", len(phenotype_df), "HP:", phenotype_df["HP"].nunique(), "OMIM:", phenotype_df["OMIM"].nunique() )
print('CTDでマッピングされるMESH-OMIM:', "マッピング数:", len(ctd), "MESH:", ctd["MESH"].nunique(), "OMIM:", ctd["OMIM"].nunique())

phenotype.hpoaのアノテーション総数: 120331 HP: 8041 OMIM: 8120
CTDでマッピングされるMESH-OMIM: マッピング数: 4051 MESH: 3330 OMIM: 4051


In [14]:
#2012AA セマンティックタイプから,T184-CUI-HPOを調査

In [15]:
print(file_4)
mrfiles = pd.read_csv(file_4, sep="\|", header=None, engine="python")
mrfiles.columns=["FIL", "DES", "FMT", "CLS", "RWS" ,"BTS", 0]
mrfiles = mrfiles.iloc[:,:-1]

#FILがファイル名, FMTが列名
mrfiles["FMT_split"] = mrfiles ["FMT"].str.split(",")

MRFILES.RRF.gz


In [16]:
#列名取得
conso_col = mrfiles[mrfiles ["FIL"] =="MRCONSO.RRF"]["FMT_split"].values.tolist()[0] #2MRCONSO.RRF の列名
mrsty_col = mrfiles[mrfiles ["FIL"] =="MRSTY.RRF"]["FMT_split"].values.tolist()[0] #3"MRSTY.RRF"の列名
print(len(conso_col), conso_col)
print(len(mrsty_col), mrsty_col)

18 ['CUI', 'LAT', 'TS', 'LUI', 'STT', 'SUI', 'ISPREF', 'AUI', 'SAUI', 'SCUI', 'SDUI', 'SAB', 'TTY', 'CODE', 'STR', 'SRL', 'SUPPRESS', 'CVF']
6 ['CUI', 'TUI', 'STN', 'STY', 'ATUI', 'CVF']


In [17]:
#3 MRSTY.RRF 	Semantic Types→セマンティックタイプ
print(file_5)
mrsty_org = pd.read_csv(file_5, sep="\|", header=None, engine="python")
mrsty_org.columns = mrsty_col + [0]
mrsty_org = mrsty_org.iloc[:,:len(mrsty_col)]
#データがあるのはCUI, TUI列のみだったので２列だけに絞る
mrsty_org = mrsty_org[["CUI", "TUI"]]
print(len(mrsty_org))

MRSTY.RRF.gz
4864162


In [18]:
#T184を持つCUI
t184_cui = mrsty_org[ mrsty_org["TUI"] == "T184"]["CUI"].values.tolist()
print("mrsty.rrf のうち T184_CUI:", len(t184_cui))

#UMLSのT184に紐づけられたCUI。HPOオントロジーではHPOとCUIが紐づけられているので、CUIを使ってT184とHPOがマッピングできる
#HPO_CUIのペアのうち、T184_CUIとペアになっているHPOを取り出す
t184_hp = sorted(list(set([ i[0] for i in hp_umls  if i[1] in t184_cui])))
print("T184_CUI 紐づけらるHPO:", len(t184_hp))

#subsetのHPOのうち T184にあたるHPOの件数
print("phenotype_dfのHPOのうち T184にあたるHPOの件数:", len(set(phenotype_df["HP"].values.tolist()) & set(t184_hp)))

mrsty.rrf のうち T184_CUI: 14281
T184_CUI 紐づけらるHPO: 216
phenotype_dfのHPOのうち T184にあたるHPOの件数: 176


In [19]:
#アノテーションデータをT184_HPOで絞り込む
t184_index = [enu for enu, i in enumerate(phenotype_df["HP"])  if i in t184_hp]
phenotype_df_t184= phenotype_df.iloc[t184_index].reset_index(drop=True)
print(len(phenotype_df_t184), "HP:", phenotype_df_t184["HP"].nunique(), "OMIM:", phenotype_df_t184["OMIM"].nunique())

7238 HP: 176 OMIM: 3509


In [20]:
print(phenotype_df_t184["OMIM"].nunique(), ctd["OMIM"].nunique())

3509 4051


In [21]:
#アノテーションデータを症状タームで限定した（T184にあたるHPOのみ）
#さらに疾患タームで限定する（MESHに置換できるOMIMのみ）
#subset

subset = pd.merge(phenotype_df_t184, ctd)
print(len(subset))
[ [i, subset[i].nunique()] for i in subset.columns]

3645


[['HP', 165], ['OMIM', 1669], ['MESH', 1431]]

In [22]:
#MESH-HPOを疾患-症状の共起データとし、症状HPOを共有する疾患ー疾患ネットワークを作る
co_occ_pair = pd.DataFrame(subset.groupby(['HP', 'MESH']).size()).reset_index()
co_occ_pair = co_occ_pair.rename(columns={0:'co_occurrence'})
co_occ_pair = co_occ_pair.sort_values(by = "co_occurrence", ascending=False).reset_index(drop=True)
print(len(co_occ_pair))
co_occ_pair["co_occurrence"].value_counts()

3368


1     3211
2      107
3       21
4       13
5        7
6        3
8        2
7        2
10       1
12       1
Name: co_occurrence, dtype: int64

In [23]:
#HPO_IDでグルーピング　同じHPOを持つMESH同士がまとまる
group = co_occ_pair.groupby("HP").agg({"MESH":list}) 

In [24]:
#HPOごとに、そのHPOを共有するMESH同士をつなげる

bm_pair = []
for mesh in tqdm(group["MESH"]):
    
    #omimの組み合わせを作る
    pairs = list(itertools.combinations(mesh, 2))
    
    #その組み合わせからペアを取り出す
    for each_pair in pairs:
        bm_pair.append(each_pair)

benchmark_pair = list(set(bm_pair))
print(len(bm_pair), "--->", len(benchmark_pair))

  0%|          | 0/165 [00:00<?, ?it/s]

182161 ---> 173069


In [25]:
#エッジリスト用データフレームにする
benchmark = pd.DataFrame(list(benchmark_pair), columns=["disease_1", "disease_2"] )

#エッジリスト読込・ネットワーク作成

import networkx as nx
bench_G = nx.from_pandas_edgelist(benchmark, "disease_1", "disease_2" )

print("benchmark:", nx.info(bench_G), "\n")

benchmark: Name: 
Type: Graph
Number of nodes: 1424
Number of edges: 165320
Average degree: 232.1910 



In [26]:
#セマンティックネットワークの再構築

In [27]:
print(file_6)
mrconso_aa_org = pd.read_csv(file_6, sep="\|", header=None, engine="python")
mrconso_aa_org.columns= conso_col + [0]
mrconso_aa_org = mrconso_aa_org.iloc[:,:len(conso_col)]

MRCONSO.RRF.aa.gz


In [28]:
print(file_7)
mrconso_ab_org = pd.read_csv(file_7, sep="\|", header=None, engine="python")
mrconso_ab_org = mrconso_ab_org.reset_index() #必要
mrconso_ab_org.columns=conso_col + [0]
mrconso_ab_org = mrconso_ab_org.iloc[:,:len(conso_col)]


MRCONSO.RRF.ab.gz


In [29]:
#mrconso_aa_orgの最後の行のデータの続きが mrconso_ab_orgの一行目にある。
#データをくっつけて一行にする
data = [i for i in mrconso_aa_org[-1:].values.tolist()[0]] +  [i for i in mrconso_ab_org[:1].values.tolist()[0]]
#[[enu,i] for enu, i in enumerate(data)]

rewrite_data = data[:3] + [str(data[3])+ str(data[18])] + data[19:-3]
print(len(rewrite_data), rewrite_data)

18 ['C1507708', 'ENG', 'S', 'L8533703', 'PF', 'S10618931', 'Y', 'A18343928', nan, '35548-7', nan, 'LNC', 'LC', '35548-7', 'Fusarium moniliforme IgG Ab [Mass/volume] in Serum', '0', 'N', nan]


In [30]:
tmp_df = pd.DataFrame(data=rewrite_data).T
tmp_df.columns=conso_col

In [31]:
#データフレーム調整
mrconso_aa = mrconso_aa_org[:-1]
mrconso_ab = mrconso_ab_org.append(tmp_df)[1:].sort_index() #mrconso_ab_org.update(df)
print(len(mrconso_aa_org), len(mrconso_aa), len(mrconso_ab_org), len(mrconso_ab) )

8683998 8683997 7859674 7859674


In [32]:
#mrconso_aa, mrconso_ab 合体
conso = pd.concat([mrconso_aa, mrconso_ab ] )
conso = conso.reset_index(drop=True)
print(len(conso))#hpo["LAT"].unique() #ENGのみ
#conso.to_csv("conso_2021AB.csv", index=False)

16543671


In [33]:
conso.to_csv("conso_2021AB.csv", index=False)

In [34]:
#CUI, HPO, STR(ターム名）

#HPO ONTOLOGY の HPO-CUI
hp_umls_df = pd.DataFrame(hp_umls, columns=["HPO", "CUI"])
#CONSO の CUI-STR(各ソースでの名前) と合わせる
conso_hp_cui = pd.merge(hp_umls_df, conso[["CUI", "STR"]] ) #CUI-STR(各ソースでの名前)
conso_hp_cui = conso_hp_cui[~conso_hp_cui.duplicated()]
len(conso_hp_cui)

273619

In [35]:
#セマンティックデータ(CUI-TUI) と合わせる
#CUI-TUI-HPO-STR(C各ソースでの名前）がつながる
semtypes = pd.merge(mrsty_org, conso_hp_cui ) 
print(len(semtypes), len(semtypes["CUI"].unique()), len(semtypes["TUI"].unique()), len(semtypes["HPO"].unique()))
semtypes[:5]

273952 12712 33 11486


,CUI,TUI,HPO,STR
0,C0000731,T033,HP:0003270,ABDOMENAREN BOLUMEN ALDAKETA/ABDOMEN HAUNDITZEA
1,C0000731,T033,HP:0003270,Břišní distenze
2,C0000731,T033,HP:0003270,Vzedmuté břicho
3,C0000731,T033,HP:0003270,Zduřené břicho
4,C0000731,T033,HP:0003270,AEndring af abdom omfang/udspiling


In [36]:
#HPO名を割り当てる
semtypes["HPO_name"] = [ont[i].name for i in semtypes["HPO"]]
semtypes["HPO_id_name"] = [ont[i].name + '(' + i + ')' for i in semtypes["HPO"]]
semtypes["CUI_STR"] = [ i[0] + '(' + i[1] + ')' for i in semtypes[["CUI", "STR"]].values.tolist() ]
semtypes = semtypes[["HPO_id_name", "HPO", "CUI", "CUI_STR","TUI"]]
print(len(semtypes))

273952


In [37]:
semtypes[:5]

,HPO_id_name,HPO,CUI,CUI_STR,TUI
0,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(ABDOMENAREN BOLUMEN ALDAKETA/ABDOMEN ...,T033
1,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(Břišní distenze),T033
2,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(Vzedmuté břicho),T033
3,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(Zduřené břicho),T033
4,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(AEndring af abdom omfang/udspiling),T033


In [38]:
semtypes[:5]

,HPO_id_name,HPO,CUI,CUI_STR,TUI
0,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(ABDOMENAREN BOLUMEN ALDAKETA/ABDOMEN ...,T033
1,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(Břišní distenze),T033
2,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(Vzedmuté břicho),T033
3,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(Zduřené břicho),T033
4,Abdominal distention(HP:0003270),HP:0003270,C0000731,C0000731(AEndring af abdom omfang/udspiling),T033


In [39]:
#TUI名を割り当てる
#https://lhncbc.nlm.nih.gov/semanticnetwork/SemanticNetworkArchive.html
#SN_2012AA - SRDEF, SRFIL
#SRFILファイルから　SRDEFファイルの列名をS取得

print(file_8)
srfil = pd.read_table(file_8, sep="\|", header=None, engine='python')
srfil["col_name"] = srfil [2].str.split(",")
srdef_col = srfil[srfil [0] =="SRDEF"]["col_name"].values.tolist()[0] 

srdef = pd.read_table("SRDEF", sep="\|", header=None, engine='python')
srdef.columns= srdef_col+[0]
srdef = srdef.iloc[:, :-1]
srdef[ (srdef["RT"] == "STY") & (srdef["UI"] == "T184")]

tui_name = srdef[ (srdef["RT"] == "STY") ][["UI", "STY/RL"]]
tui_name = tui_name.rename(columns={"UI":"TUI", "STY/RL":"TUI_name"})


SRFIL


In [40]:
df = pd.merge(semtypes, tui_name)
print(len(df))
[ [i, df[i].nunique()] for i in df.columns]

273952


[['HPO_id_name', 11486],
 ['HPO', 11486],
 ['CUI', 12712],
 ['CUI_STR', 261353],
 ['TUI', 33],
 ['TUI_name', 33]]

In [41]:
#集計
shukei_df = df.groupby(["TUI", "TUI_name"]).agg({'CUI':list, "HPO":list})

shukei_df["CUI"] = [list(set(i)) for i in shukei_df["CUI"]]
shukei_df["HPO"] = [list(set(i)) for i in shukei_df["HPO"]]

shukei_df["CUI_num"] = [len(i) for i in shukei_df["CUI"]]
shukei_df["HPO_num"] = [len(i) for i in shukei_df["HPO"]]

shukei_df = shukei_df.reset_index()
len(shukei_df)

33

In [42]:
cui_freq = shukei_df[["TUI", "TUI_name", "CUI_num"]].sort_values("CUI_num", ascending=False).reset_index(drop=True)
cui_freq[:10]

,TUI,TUI_name,CUI_num
0,T033,Finding,5164
1,T190,Anatomical Abnormality,3005
2,T047,Disease or Syndrome,2147
3,T019,Congenital Abnormality,951
4,T191,Neoplastic Process,517
5,T046,Pathologic Function,383
6,T184,Sign or Symptom,218
7,T048,Mental or Behavioral Dysfunction,127
8,T037,Injury or Poisoning,39
9,T020,Acquired Abnormality,34


In [43]:
hpid_freq= shukei_df[["TUI", "TUI_name", "HPO_num"]].sort_values("HPO_num", ascending=False).reset_index(drop=True)
hpid_freq[:11]

,TUI,TUI_name,HPO_num
0,T033,Finding,4644
1,T190,Anatomical Abnormality,2964
2,T047,Disease or Syndrome,2069
3,T019,Congenital Abnormality,944
4,T191,Neoplastic Process,498
5,T046,Pathologic Function,371
6,T184,Sign or Symptom,216
7,T048,Mental or Behavioral Dysfunction,120
8,T037,Injury or Poisoning,38
9,T020,Acquired Abnormality,34
